<a href="https://colab.research.google.com/github/Apoak/Deep-Learning-Projects/blob/main/Batching_and_Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Lab 3.1: Batching and Regularization

In this lab you will learn how to set up a dataset to be processed in batches, rather than processing the entire dataset in each training iteration, and explore neural network regularization.

In [ ]:
import numpy as np
import torch
import sklearn
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
!pip install scikit-learn ucimlrepo mlxtend
from ucimlrepo import fetch_ucirepo

# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

# metadata
print(adult.metadata)

# variable information
print(adult.variables)

In [ ]:
X.columns

In [ ]:
y = y['income'].map({'<=50K':0,'<=50K.':0,'>50K':1,'>50K.':1})

In [ ]:
X = X[['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']]

In [ ]:
y = y.values
X = X.values.astype('float64')

To make the learning algorithm work more smoothly, we we will subtract the mean of each feature.

Here `np.mean` calculates a mean, and `axis=0` tells NumPy to calculate the mean over the rows (calculate the mean of each column).

In [ ]:
X -= np.mean(X,axis=0)

Now we will convert our `X` and `y` arrays to torch Tensors.

In [ ]:
X = torch.tensor(X).float()
y = torch.tensor(y).long()

bad = X.isna().any(axis=1)
X = X[~bad]
y = y[~bad]

### Exercises

1. Divide the data into train and test splits.
2. Create a neural network for this dataset.
3. Use `TensorDataset` and `DataLoader` to batch the dataset during training.  
4. Use `weight_decay` parameter to `optim.SGD` to introduce L2 regularization during training. Evaluate the effect of regularization on test set accuracy.

In [ ]:
dataset = TensorDataset(X, y)

In [ ]:
# numba 1
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size= .1, train_size= .9, random_state= 42)

In [ ]:
# numba 2
multilayer_model = torch.nn.Sequential(
    torch.nn.Linear(6,100),
    torch.nn.ReLU(),
    torch.nn.Linear(100,2),
    # two inputs, three outputs
)

lr = 1e-2
loss_fn = torch.nn.CrossEntropyLoss()
opt = torch.optim.SGD(multilayer_model.parameters(), lr=lr, weight_decay= 0.001)

In [ ]:
# numba 3 MIGHT NEED TO DO THE GRADIENT INITIALIZATION THING FROM LAB 3.1
batch_size = 32 # 32 in homework
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

for epoch in range(10):  # Example: 3 epochs
  print(f"Epoch {epoch + 1}")
  for batch_idx, (batch_X, batch_y) in enumerate(dataloader):
    opt.zero_grad() # zero out the gradients
    z = multilayer_model(X) # compute z values
    loss = loss_fn(z,y) # compute loss
    loss.backward() # compute gradients
    opt.step()
  print(f'epoch {epoch}: loss is {loss.item()}')


In [ ]:
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# numba 4
multilayer_model.eval()
correct = 0
total = 0

with torch.no_grad():  # Disable gradient computation for testing
    for batch_X, batch_y in test_loader:
        outputs = multilayer_model(batch_X)
        _, predicted = torch.max(outputs, 1)  # Get predicted class
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")


**Response:** My loss is being outputed as nan so this is speculative. The result of the weight decay in regularization will reduce variance in the fit to the training data. So, this should make the model more general and increase the test accuracy on the testing data.